In [48]:
import os
import random
import math
import shutil
import time
import warnings
import soundfile
import csv
import torchvision

import librosa
import argparse
import numpy as np

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchaudio
from torchaudio import transforms

import librosa
import os
import shutil
import pandas as pd
import pickle as pkl 
from PIL import Image
import lmdb
import random

from torch.utils.data import *
import torchvision

## Preprocessing for test audio

In [49]:
%cd

/home/ubuntu


In [51]:
def extract_spectrogram_test(values, clip):
    sampling_rate = 44100

    num_channels = 3
    window_sizes = [25, 50, 100]
    hop_sizes = [10, 25, 50]
    centre_sec = 2.5

    specs = []
    for i in range(num_channels):
        window_length = int(round(window_sizes[i]*sampling_rate/1000))
        hop_length = int(round(hop_sizes[i]*sampling_rate/1000))

        clip = torch.Tensor(clip)
        spec = torchaudio.transforms.MelSpectrogram(sample_rate=sampling_rate, n_fft=4410, win_length=window_length, hop_length=hop_length, n_mels=128)(clip)
        eps = 1e-6
        spec = spec.numpy()
        spec = np.log(spec+ eps)
        spec = np.asarray(torchvision.transforms.Resize((128, 250))(Image.fromarray(spec)))
        specs.append(spec)
    new_entry = {}
    new_entry["audio"] = clip.numpy()
    new_entry["values"] = np.array(specs)
#         new_entry["target"] = data["target"]
    values.append(new_entry)

def extract_features(audios):
    data_dir = 'DataSci251_FinalProject/Test_Audio'
    sampling_rate = 44100
    
    audio_names = audios
    file_labels = []
    values = []
    for audio in audio_names:
        clip, sr = librosa.load("{}/{}".format(data_dir, audio), sr=sampling_rate)
#         entries = audios.loc[audios["filename"]==audio].to_dict(orient="records")
        extract_spectrogram_test(values, clip)
        print("Finished audio {}".format(audio))
        file_labels += [audio]
    return values, file_labels

In [52]:
store_dir = 'DataSci251_FinalProject/DataSet/ESC-50-master/store_spectograms_test'
test_audios = os.listdir('DataSci251_FinalProject/Test_Audio/')

testing_values, labels = extract_features(test_audios)
with open("{}testing128mel{}.pkl".format(store_dir, 1),"wb") as handler:
    pkl.dump(testing_values, handler, protocol=pkl.HIGHEST_PROTOCOL)

Finished audio audio_222002.wav
Finished audio audio_222026.wav
Finished audio audio_222051.wav
Finished audio audio_222006.wav
Finished audio audio_cough_8.wav
Finished audio audio_221654.wav
Finished audio audio_222151.wav
Finished audio audio_222126.wav
Finished audio audio_222319.wav
Finished audio audio_221957.wav
Finished audio audio_221947.wav
Finished audio audio_222036.wav
Finished audio audio_221729.wav
Finished audio audio_221629.wav
Finished audio audio_221251.wav
Finished audio audio_222304.wav
Finished audio audio_cough_19.wav
Finished audio audio_221749.wav
Finished audio audio_222021.wav
Finished audio audio_221927.wav
Finished audio audio_221634.wav
Finished audio audio_222309.wav
Finished audio audio_221236.wav
Finished audio audio_222409.wav
Finished audio audio_221739.wav
Finished audio audio_222041.wav
Finished audio audio_222324.wav
Finished audio audio_221221.wav
Finished audio audio_222429.wav
Finished audio audio_221709.wav
Finished audio audio_221206.wav
Finis

In [54]:
class MelSpectrogram(object):
    def __init__(self, bins, mode, dataset):
        self.window_length = [25, 50, 100]
        self.hop_length = [10, 25, 50]
        self.fft = 4410
        self.melbins = bins
        self.mode = mode
        self.sr = 44100
        self.length = 120
    def __call__(self, value):
        sample = value
        limits = ((-2, 2), (0.9, 1.2))

        if self.mode=="train":
            pitch_shift = np.random.randint(limits[0][0], limits[0][1] + 1)
            time_stretch = np.random.random() * (limits[1][1] - limits[1][0]) + limits[1][0]
            new_audio = librosa.effects.time_stretch(y = librosa.effects.pitch_shift(y = sample, sr = self.sr, n_steps = pitch_shift), rate = time_stretch)
        else:
            pitch_shift = 0
            time_stretch = 1
            new_audio = sample
        specs = []
        for i in range(len(self.window_length)):
            clip = torch.Tensor(new_audio)

            window_length = int(round(self.window_length[i]*self.sr/1000))
            hop_length = int(round(self.hop_length[i]*self.sr/1000))
            spec = torchaudio.transforms.MelSpectrogram(sample_rate=self.sr, n_fft=self.fft, win_length=window_length, hop_length=hop_length, n_mels=self.melbins)(clip)
            eps = 1e-6
            spec = spec.numpy()
            spec = np.log(spec+ eps)
            spec = np.asarray(torchvision.transforms.Resize((128, self.length))(Image.fromarray(spec)))
            specs.append(spec)
        specs = np.array(specs).reshape(-1, 128, self.length)
        specs = torch.Tensor(specs)
        return specs


class AudioDatasetTest(Dataset):
    def __init__(self, pkl_dir, dataset_name, labels, transforms=None):
        self.transforms = transforms
        self.labels = labels
        self.data = []
        self.length = 250
        with open(pkl_dir, "rb") as f:
            self.data = pkl.load(f)
    def __len__(self):
        if self.transforms.mode == "train":
            return 2*len(self.data)
        else:
            return len(self.data)
    def __getitem__(self, idx):
        if idx >= len(self.data):
            new_idx = idx - len(self.data)
            entry = self.data[new_idx]
            if self.transforms:
                values = self.transforms(entry["audio"])
        else:
            entry = self.data[idx]
            values = torch.Tensor(entry["values"].reshape(-1, 128, self.length))
#         target = torch.LongTensor([entry["target"]])
#         target = torch.LongTensor(500)
            label = self.labels[idx]
        return (values, label)
    

def fetch_dataset(pkl_dir, dataset_name, labels, mode):
    transforms = MelSpectrogram(128, mode, dataset_name)
    dataset = AudioDatasetTest(pkl_dir, dataset_name, labels, transforms=transforms)
    dataloader = DataLoader(dataset,shuffle=False, batch_size=1, num_workers=1)
    return dataloader

## Loading and testing models

### Best densenet

In [55]:
model_load = torch.jit.load('DataSci251_FinalProject/pt_Files/best_densenet.pt')

In [56]:
data_dir = 'DataSci251_FinalProject/DataSet/ESC-50-master/store_spectograms_test'
test_dataloader = fetch_dataset("{}testing128mel{}.pkl".format(data_dir, 1), "ESC",labels, 'test')

In [58]:
model_load.eval()
cough_count = 0
with torch.no_grad():
    for batch_idx, data in enumerate(test_dataloader):

        outputs = model_load(data[0])
#         print(outputs)
        _, predicted = torch.max(outputs.data, 1)
        
        pred_int = int(predicted)
        print('file name: ', data[1], ' label: ', pred_int)
        if pred_int == 24:
            cough_count += 1
            print('COUGH')

file name:  ('audio_222002.wav',)  label:  15
file name:  ('audio_222026.wav',)  label:  48
file name:  ('audio_222051.wav',)  label:  46
file name:  ('audio_222006.wav',)  label:  15
file name:  ('audio_cough_8.wav',)  label:  24
COUGH
file name:  ('audio_221654.wav',)  label:  31
file name:  ('audio_222151.wav',)  label:  15
file name:  ('audio_222126.wav',)  label:  15
file name:  ('audio_222319.wav',)  label:  48
file name:  ('audio_221957.wav',)  label:  48
file name:  ('audio_221947.wav',)  label:  31
file name:  ('audio_222036.wav',)  label:  48
file name:  ('audio_221729.wav',)  label:  48
file name:  ('audio_221629.wav',)  label:  33
file name:  ('audio_221251.wav',)  label:  30
file name:  ('audio_222304.wav',)  label:  33
file name:  ('audio_cough_19.wav',)  label:  24
COUGH
file name:  ('audio_221749.wav',)  label:  44
file name:  ('audio_222021.wav',)  label:  48
file name:  ('audio_221927.wav',)  label:  27
file name:  ('audio_221634.wav',)  label:  17
file name:  ('audio

In [59]:
cough_count

10

Model identified 10/20 coughs.

## Densenet 1

In [67]:
model_load = torch.jit.load('DataSci251_FinalProject/pt_Files/densenet1.pt')

In [68]:
model_load.eval()
cough_count = 0
with torch.no_grad():
    for batch_idx, data in enumerate(test_dataloader):

        outputs = model_load(data[0])
#         print(outputs)
        _, predicted = torch.max(outputs.data, 1)
        
        pred_int = int(predicted)
        print('file name: ', data[1], ' label: ', pred_int)
        if pred_int == 24:
            cough_count += 1
            print('COUGH')

file name:  ('audio_222002.wav',)  label:  31
file name:  ('audio_222026.wav',)  label:  48
file name:  ('audio_222051.wav',)  label:  46
file name:  ('audio_222006.wav',)  label:  31
file name:  ('audio_cough_8.wav',)  label:  48
file name:  ('audio_221654.wav',)  label:  31
file name:  ('audio_222151.wav',)  label:  29
file name:  ('audio_222126.wav',)  label:  29
file name:  ('audio_222319.wav',)  label:  31
file name:  ('audio_221957.wav',)  label:  31
file name:  ('audio_221947.wav',)  label:  33
file name:  ('audio_222036.wav',)  label:  48
file name:  ('audio_221729.wav',)  label:  48
file name:  ('audio_221629.wav',)  label:  33
file name:  ('audio_221251.wav',)  label:  31
file name:  ('audio_222304.wav',)  label:  33
file name:  ('audio_cough_19.wav',)  label:  24
COUGH
file name:  ('audio_221749.wav',)  label:  44
file name:  ('audio_222021.wav',)  label:  48
file name:  ('audio_221927.wav',)  label:  49
file name:  ('audio_221634.wav',)  label:  37
file name:  ('audio_22230

In [65]:
cough_count

16

Model identified 16/20 coughs.

## Densenet 2

In [72]:
model_load = torch.jit.load('DataSci251_FinalProject/pt_Files/densenet2.pt')

In [70]:
model_load.eval()
cough_count = 0
with torch.no_grad():
    for batch_idx, data in enumerate(test_dataloader):

        outputs = model_load(data[0])
#         print(outputs)
        _, predicted = torch.max(outputs.data, 1)
        
        pred_int = int(predicted)
        print('file name: ', data[1], ' label: ', pred_int)
        if pred_int == 24:
            cough_count += 1
            print('COUGH')

file name:  ('audio_222002.wav',)  label:  15
file name:  ('audio_222026.wav',)  label:  48
file name:  ('audio_222051.wav',)  label:  46
file name:  ('audio_222006.wav',)  label:  15
file name:  ('audio_cough_8.wav',)  label:  24
COUGH
file name:  ('audio_221654.wav',)  label:  31
file name:  ('audio_222151.wav',)  label:  15
file name:  ('audio_222126.wav',)  label:  15
file name:  ('audio_222319.wav',)  label:  48
file name:  ('audio_221957.wav',)  label:  48
file name:  ('audio_221947.wav',)  label:  31
file name:  ('audio_222036.wav',)  label:  48
file name:  ('audio_221729.wav',)  label:  48
file name:  ('audio_221629.wav',)  label:  33
file name:  ('audio_221251.wav',)  label:  30
file name:  ('audio_222304.wav',)  label:  33
file name:  ('audio_cough_19.wav',)  label:  24
COUGH
file name:  ('audio_221749.wav',)  label:  44
file name:  ('audio_222021.wav',)  label:  48
file name:  ('audio_221927.wav',)  label:  27
file name:  ('audio_221634.wav',)  label:  17
file name:  ('audio

In [71]:
cough_count

10

## Resnet

In [73]:
model_load = torch.jit.load('DataSci251_FinalProject/pt_Files/best_resnet.pt')

In [74]:
model_load.eval()
cough_count = 0
with torch.no_grad():
    for batch_idx, data in enumerate(test_dataloader):

        outputs = model_load(data[0])
#         print(outputs)
        _, predicted = torch.max(outputs.data, 1)
        
        pred_int = int(predicted)
        print('file name: ', data[1], ' label: ', pred_int)
        if pred_int == 24:
            cough_count += 1
            print('COUGH')

file name:  ('audio_222002.wav',)  label:  15
file name:  ('audio_222026.wav',)  label:  25
file name:  ('audio_222051.wav',)  label:  46
file name:  ('audio_222006.wav',)  label:  31
file name:  ('audio_cough_8.wav',)  label:  24
COUGH
file name:  ('audio_221654.wav',)  label:  31
file name:  ('audio_222151.wav',)  label:  29
file name:  ('audio_222126.wav',)  label:  15
file name:  ('audio_222319.wav',)  label:  31
file name:  ('audio_221957.wav',)  label:  29
file name:  ('audio_221947.wav',)  label:  31
file name:  ('audio_222036.wav',)  label:  31
file name:  ('audio_221729.wav',)  label:  27
file name:  ('audio_221629.wav',)  label:  31
file name:  ('audio_221251.wav',)  label:  30
file name:  ('audio_222304.wav',)  label:  28
file name:  ('audio_cough_19.wav',)  label:  24
COUGH
file name:  ('audio_221749.wav',)  label:  13
file name:  ('audio_222021.wav',)  label:  31
file name:  ('audio_221927.wav',)  label:  32
file name:  ('audio_221634.wav',)  label:  15
file name:  ('audio

In [75]:
cough_count

19

Identified 19/20 coughs!